In [4]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python
!export tavily_api_key=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

Note: you may need to restart the kernel to use updated packages.


In [5]:
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="..."

import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()


os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Las Palmas de Gran Canaria")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Las Palmas De Gran Canaria', 'region': 'Canarias', 'country': 'Spain', 'lat': 28.1, 'lon': -15.42, 'tz_id': 'Atlantic/Canary', 'localtime_epoch': 1718614710, 'localtime': '2024-06-17 9:58'}, 'current': {'last_updated_epoch': 1718613900, 'last_updated': '2024-06-17 09:45', 'temp_c': 19.4, 'temp_f': 66.8, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 11.6, 'wind_kph': 18.7, 'wind_degree': 24, 'wind_dir': 'NNE', 'pressure_mb': 1018.0, 'pressure_in': 30.05, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 74, 'cloud': 23, 'feelslike_c': 19.4, 'feelslike_f': 66.8, 'windchill_c': 19.4, 'windchill_f': 66.8, 'heatindex_c': 19.4, 'heatindex_f': 66.8, 'dewpoint_c': 14.7, 'dewpoint_f': 58.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 5.0, 'gust_mph': 13.4, 'gust_kph': 21.5}}"}, {'url': 'https://www.metoffice.gov.uk/weather/forecast/esvncvn3w', '

In [8]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [10]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hola!")])
response.content

'¡Hola! ¿Cómo puedo ayudarte hoy?'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use .bind_tools to give the language model knowledge of these tools



In [11]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the content field as well as the tool_calls field.



In [12]:
response = model_with_tools.invoke([HumanMessage(content="Hola!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: ¡Hola! ¿Cómo puedo ayudarte hoy?
ToolCalls: []


In [13]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Las Palmas de Gran Canaria?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Las Palmas de Gran Canaria'}, 'id': 'call_71iz197ElvEFP1e0PgKJokZU'}]


## initalize the agent with the LLM and the tools. ##

In [14]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent ##

In [15]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='8f8c6a95-a039-4b90-8d42-aade9a338198'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a056aa39-63a9-400e-8fb9-21df6eb5a977-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [16]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='f4b7ddc2-0610-4927-a1ee-3aaf8b83777e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ostYg8rewjHNo1DFJFxFjcoj', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2104622f-2ef2-40fe-aafe-eaa56dfd6399-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ostYg8rewjHNo1DFJFxFjcoj'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America

## Streaming Messages ##
We've seen how the agent can be called with .invoke to get back a final response. If the agent is executing multiple steps, that may take a while. In order to show intermediate progress, we can stream back messages as they occur.

In [17]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Dubai?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_M9Xv5aOdB2uaorbp45Q8dkuh', 'function': {'arguments': '{\n  "query": "current weather in Dubai"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-250179ac-374a-4c39-ba4c-ebc9453fc5e3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Dubai'}, 'id': 'call_M9Xv5aOdB2uaorbp45Q8dkuh'}], usage_metadata={'input_tokens': 88, 'output_tokens': 22, 'total_tokens': 110})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Dubai\', \'region\': \'Dubai\', \'country\': \'United Arab Emirates\', \'lat\': 25.25, \'lon\': 55.28, \'tz_id\': \'Asia/Dubai\', \'localtime

In [19]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

I|'m| sorry|,| I| can|'t| provide| real|-time| information| such| as| weather|.| However|,| you| can| use| your| preferred| weather| checking| platform| or| search| engine| to| get| the| current| weather| in| San| Francisco|.|

## Adding in memory ##

In [20]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello, Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 85, 'total_tokens': 97}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-82fa0413-4a32-4578-af87-3e150cc484bd-0', usage_metadata={'input_tokens': 85, 'output_tokens': 12, 'total_tokens': 97})]}}
----


In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob. How can I assist you further?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 109, 'total_tokens': 122}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-335aff25-c711-4be6-93d9-233ca11171c9-0', usage_metadata={'input_tokens': 109, 'output_tokens': 13, 'total_tokens': 122})]}}
----


In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="I live in Las Palmas GC")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="That's great, Bob! Las Palmas de Gran Canaria is a beautiful place. How can I assist you further?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 136, 'total_tokens': 162}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dca3efdc-6956-47bf-bbff-e57b44875136-0', usage_metadata={'input_tokens': 136, 'output_tokens': 26, 'total_tokens': 162})]}}
----


In [23]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="How far is Tenerife from whether I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZsLLKDbhnX9I09ksDHz2AACW', 'function': {'arguments': '{\n  "query": "Distance from Las Palmas GC to Tenerife"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 180, 'total_tokens': 208}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e2f8278d-493d-4641-8fdd-0075526ce998-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Distance from Las Palmas GC to Tenerife'}, 'id': 'call_ZsLLKDbhnX9I09ksDHz2AACW'}], usage_metadata={'input_tokens': 180, 'output_tokens': 28, 'total_tokens': 208})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.rome2rio.com/s/Tenerife-Canary-Islands-Spain/Las-Palmas-de-Gran-Canaria", "content": "The distance between Tenerife and Las Palmas de Gran Canaria is 136 km. How do I 

In [24]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="How much does a flight ticket cost between both cities?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3FjTopo88WTnRtfG5ydegm3U', 'function': {'arguments': '{"query": "Flight cost from Las Palmas GC to Tenerife"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 535, 'total_tokens': 561}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-62143449-1c18-47ea-a1e0-b98f7c455b7b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Flight cost from Las Palmas GC to Tenerife'}, 'id': 'call_3FjTopo88WTnRtfG5ydegm3U'}], usage_metadata={'input_tokens': 535, 'output_tokens': 26, 'total_tokens': 561})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.kayak.com/flight-routes/Las-Palmas-de-Gran-Canaria-LPA/Granadilla-Tenerife-Sur-TFS", "content": "The cheapest month for flights from Las Palmas de Gran Canaria to Gran